In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

# Lab 9: Pre-Midterm Review

In this **optional lab**, we will review several concepts from earlier in the semester:
1. SQL
1. Pandas
1. Regression/OLS

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** at the top of this notebook.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import math
import matplotlib.pyplot as plt
import lab09_utils
from pathlib import Path
import sqlalchemy
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline
!python3 lab09_utils.py

# Part 1: SQL

## Loading the data

For this problem, we'll look at some tropical storm data. The cells below will import the data from a provided CSV and convert it into a SQL server. The data comes from [this Wikipedia article](https://en.wikipedia.org/wiki/Tropical_cyclones_by_year).

In [2]:
db_path = Path('./data/sql/trop_storms.db')

In [3]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

In [4]:
inspector.get_table_names()

In the cell below, we've displayed the first 5 columns of this SQL table for reference.

In [5]:
query = """
SELECT *
FROM trop_storm
GROUP BY Basin
LIMIT 5
"""
pd.read_sql(query, engine)

## Question 1

### Part 1

We want to find how many storms have happened in each region in the past. Write a query to do this, and store the total number of tropical storms in each region in a column called `Tot_Tropical`.

<!--
BEGIN QUESTION
name: q1a
-->

In [6]:
query = """
SELECT ..., ...
FROM trop_storm
GROUP BY ...
ORDER BY ...
"""


res_q1a = pd.read_sql(query, engine)
res_q1a

In [ ]:
grader.check("q1a")

### Part 2

While this seems like some useful data, there are some problems. Look at [this Wikipedia article](https://en.wikipedia.org/wiki/Tropical_cyclones_by_year). We can see that for some years, only some regions have data reported. 

Starting at 1968, we can see that data is being reported for all the current regions. Create a query that outputs a result with the same structure as the previous one, but this time, only select data from after the `Year` 1968. Additionally, don't include the row where the `Basin` is Worldwide.

**Hint:** Consider using a subquery.

<!--
BEGIN QUESTION
name: q1b
-->

In [8]:
query = """
SELECT ..., ...
FROM (...)
GROUP BY ...
ORDER BY ...
"""

res_q1b = pd.read_sql(query, engine)
res_q1b

In [ ]:
grader.check("q1b")

## Question 2

### Part 1

We can see in the data that these tropical storms cause loss of life and other damages. While this is a very inexact method of calculating the damages, let's use the average number of deaths per storm as a proxy for how dangerous these storms are.

Write a query that calculates the average number of deaths per storm in each of the basins. Let the column with this average be called `Deaths per storm`. Make sure that the rows are sorted by the basin name.

<!--
BEGIN QUESTION
name: q2a
-->

In [10]:
query = """
SELECT ..., ...
FROM trop_storm
GROUP BY ...
"""


res_q2a = pd.read_sql(query, engine)
res_q2a

In [ ]:
grader.check("q2a")

### Part 2

We can see in the output above that the North Indian Ocean Basin seems to a high rate of deaths per storm; in fact, it's twice as high as the next highest death rate. Let's look into that a bit more. In the following cell, write a query that selects all data in every the row from the North Indian Ocean Basin. Sort the data in decreasing order of number of deaths.

<!--
BEGIN QUESTION
name: q2b
-->

In [12]:
query = """
SELECT ...
FROM trop_storm
WHERE ...
ORDER BY ...
LIMIT 5
"""


res_q2b = pd.read_sql(query, engine)
res_q2b

In [ ]:
grader.check("q2b")

### Part 3

Do some research into the year with the most deaths. What happened this year that caused so many deaths?

<!--
BEGIN QUESTION
name: q2c
-->

_Type your answer here, replacing this text._

### Part 4

What are some problems with using the number of deaths per storm as a proxy for the danger of tropical storms in each region?
<!--
BEGIN QUESTION
name: q2d
-->

_Type your answer here, replacing this text._

# Part 2: Pandas, Feature Engineering, and Linear Regression

## Question 3: Loading the Diabetes Dataset

### Part 1

To begin, let's load the provided diabetes dataset.

The dataset is contained in the file `./diabetes.txt`. Inspect the file, and use `pd.read_csv` to store it as a DataFrame in the variable `diabetes`.

**Hint:** look into the `sep` argument for `read_csv`.
<!--
BEGIN QUESTION
name: q3a
-->

In [15]:
diabetes = ...
diabetes.head()

### Part 2
Now, let's separate the data into the design matrix $X$ and the response vector $y$.

Note that $y$ is the response of interest, a quantitative measure of disease progression one year after baseline.

<!--
BEGIN QUESTION
name: q3b
-->

In [16]:
y = ...
X = ...
X.head(10)

In [ ]:
grader.check("q3b")

## Question 4: Pandas & Feature Engineering

Looking at our data, a few possible problems appear. We see that the `SEX` column takes on values "F" and "M", which we will need to convert to a numeric value. In addition, we can see that the values in each column have varying ranges and magnitudes. In order to have a similar scale and range for each of the features, let's normalize them.

If we look at row 5 in the dataset, we can also see something peculiar.

In [18]:
X.iloc[5]

While most of the rows have a number in their `S4` column, this one doesn't. There must have been a mistake when processing the data! Lets try to gauge the scope of this error.

### Part 1
Use a Pandas method to find the number of cells affected by this problem and save it in the variable `res_q4a`.
<!--
BEGIN QUESTION
name: q4a
-->

In [19]:
res_q4a = ...
res_q4a

In [ ]:
grader.check("q4a")

Clearly this is a problem. If we're trying to run a regression on this dataset, we'll need to have numbers in the columns, not letters. Let's think of some ways we could solve this.

### Part 2

Assign `res_q4b` to a Numpy array containing the number of each of the correct options.

We could:
1. Remove rows with F in the `S4` column.
1. Remove the whole `S4` column.
1. Impute the mean over the `S4` column.
1. Do nothing
1. Impute the most common value (mode) over the `S4` column.


<!--
BEGIN QUESTION
name: q4b
-->

In [21]:
res_q4b = ...

In [ ]:
grader.check("q4b")

### Part 3

In the markdown cell below, explain the pros and cons of each of the options for 4b.

<!--
BEGIN QUESTION
name: q4c
-->

_Type your answer here, replacing this text._

### Part 4: Fixing the Data

Fortunately, a friend who worked with processing this data happens to know what the probem was. It appears that the value `'2'` was replaced with the value `'F'` in the `S4` column. In the following cell, correct this error for the `X` dataframe.
<!--
BEGIN QUESTION
name: q4d
-->

In [23]:
...

In [ ]:
grader.check("q4d")

### Part 5
Now, let's write a function to one-hot-encode the `SEX` column. Consider using `pd.get_dummies` or `sklearn.feature_extraction.DictVectorizer`.

**Hint:** Currently, some the numerical data is of type `str`, so you will need to convert the appropriate column(s) to `int` first.
<!--
BEGIN QUESTION
name: q4e
-->

In [25]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input data.
    
    Parameters
    -----------
    data: a dataframe that may include non-numerical features
    
    Returns
    -----------
    A one-hot encoded dataframe that only contains numeric features
    
    """
    ...
    return ...
one_hot_X = one_hot_encode(X)
one_hot_X.head()

### Part 6
Do we need to drop any of the columns? Think about what this means with respect to a bias column.
```
BEGIN QUESTION:
name: q4f
```

_Type your answer here, replacing this text._

### Part 7
Finally, let's write a function normalize each of the columns of `X` so that they have mean 0 and standard deviation 1. Make sure that your function also normalizes `y`!
```
BEGIN QUESTION:
name: q2g
```

In [26]:
def normalize(X, y):
    """
    Return the normalized dataframe of our input data.
    
    Parameters
    -----------
    data: a dataframe with numerical features.
    
    Returns
    -----------
    A dataframe that contains our input data, normalized so each column has mean 0 and standard deviation 1.
    
    """
    ...

normalized_X, normalized_y = normalize(one_hot_X, y)
normalized_X.head()

## Question 5: Linear Regression

Now that our design matrix is of the right form, we can start to make our regressions.

In the cells below, we've added the loss functions, linear model, and function that minimizes the loss from Lab 8.

In [27]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    
    Parameters
    -----------
    thetas: a 1D vector representing the parameters of our model ([theta1, theta2, ...])
    X: a 2D dataframe of numeric features (may also be a 2D numpy array)
    
    Returns
    -----------
    A 1D vector representing the linear combination of thetas and features as defined above.
    """
    return X.dot(thetas)

In [28]:
from scipy.optimize import minimize

def l1(y, y_hat):
    return np.abs(y - y_hat)

def l2(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, X, y):
    """
    Minimize the average loss calculated from using different theta vectors, and 
    estimate the optimal theta for the model.
    
    Parameters
    -----------
    loss_function: either the squared or absolute loss functions defined above
    model: the model (as defined in Question 1b)
    X: a 2D dataframe (or numpy array) of numeric features (one-hot encoded)
    y: a 1D vector of tip amounts
    
    Returns
    -----------
    The estimate for the optimal theta vector that minimizes our loss
    """
    return minimize(lambda theta: loss_function(model(theta, X), y).mean(), x0=np.random.rand(X.shape[1]))['x']

### Part 1: L2 Regression
Let's find the model that minimizes the L2 loss using the functions provided above.
<!--
BEGIN QUESTION
name: q5a
-->

In [29]:
res_q5a = ...

In [ ]:
grader.check("q5a")

### Part 2: L1 Regression
Now, let's find the model that minimizes the L1 loss using the functions provided above.
<!--
BEGIN QUESTION
name: q5b
-->

In [31]:
res_q5b = ...

In [ ]:
grader.check("q5b")

### Part 2: Conclusion

So far, we've created two different linear models using different loss functions. In the next few weeks we'll see some ways that you can improve and evaluate your model.

# Solutions

Congrats on finishing the lab. This lab will not be graded, but make sure to come ask for help if you have any questions! 

You don't need to submit this lab, so we haven't created a Gradescope assignment. However, the system will still generate the submission zip file, but you may ignore that!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)